In [1]:
# Dependencies
from citipy import citipy
from random import uniform
import json
import requests
from pprint import pprint
from config import api_key

In [2]:
# Retrive 1000 citites objects from citipy
cities = []

while len(cities) <1000:
    latitude = uniform(90, -90)
    longitude = uniform(180, -180)
    new_city = citipy.nearest_city(latitude, longitude)
    if new_city not in cities:
        cities.append(new_city)    

In [3]:
# Building query and retreiving API query
city_info=[]
num_request = 1000
for city in cities:
    num_request-=1
    base_url = "http://api.openweathermap.org/data/2.5/weather?q="
    query_url = (f"{base_url}{city.city_name},{city.country_code}&units=metric&appid={api_key}")
    weather_response= requests.get(query_url)
    city_info.append(weather_response.json())
    print(f"Retrieving data for {city.city_name} in {city.country_code}.\n {num_request} requests left.")

Retrieving data for bluff in nz.
 999 requests left.
Retrieving data for ushuaia in ar.
 998 requests left.
Retrieving data for castro in cl.
 997 requests left.
Retrieving data for usak in tr.
 996 requests left.
Retrieving data for khatanga in ru.
 995 requests left.
Retrieving data for cidreira in br.
 994 requests left.
Retrieving data for saskylakh in ru.
 993 requests left.
Retrieving data for airai in pw.
 992 requests left.
Retrieving data for malakal in sd.
 991 requests left.
Retrieving data for petropavlovsk-kamchatskiy in ru.
 990 requests left.
Retrieving data for punta arenas in cl.
 989 requests left.
Retrieving data for tuktoyaktuk in ca.
 988 requests left.
Retrieving data for port elizabeth in za.
 987 requests left.
Retrieving data for albany in au.
 986 requests left.
Retrieving data for bambous virieux in mu.
 985 requests left.
Retrieving data for rikitea in pf.
 984 requests left.
Retrieving data for busselton in au.
 983 requests left.
Retrieving data for palabu

In [4]:
# Pretty Print response
pprint(city_info)

[{'base': 'stations',
  'clouds': {'all': 100},
  'cod': 200,
  'coord': {'lat': -46.6, 'lon': 168.3333},
  'dt': 1616736264,
  'id': 2206939,
  'main': {'feels_like': 14.79,
           'humidity': 79,
           'pressure': 1016,
           'temp': 16.67,
           'temp_max': 16.67,
           'temp_min': 16.67},
  'name': 'Bluff',
  'rain': {'1h': 0.47},
  'sys': {'country': 'NZ',
          'id': 2008959,
          'sunrise': 1616698585,
          'sunset': 1616741298,
          'type': 3},
  'timezone': 46800,
  'visibility': 10000,
  'weather': [{'description': 'light rain',
               'icon': '10d',
               'id': 500,
               'main': 'Rain'}],
  'wind': {'deg': 331, 'gust': 7.15, 'speed': 4.02}},
 {'base': 'stations',
  'clouds': {'all': 90},
  'cod': 200,
  'coord': {'lat': -54.8, 'lon': -68.3},
  'dt': 1616736024,
  'id': 3833367,
  'main': {'feels_like': -1.18,
           'humidity': 87,
           'pressure': 1006,
           'temp': 5,
           'temp_max